## Credit card clients
#### Objective
develop a predictive model for whether an account holder will default next month, given demographic and histocial data.

In [1]:
# load libraries
import pandas as pd
import numpy as np
from numpy.random import default_rng

In [2]:
# read in the data
df = pd.read_excel("Data/credit_card_clients.xls")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


### data dictionary (categorical variables):
- Education (1= graduate school, 2 = Univeristy, 3= high school, 4= other)
- sex (1= male, 2= female)
- marriage (1 =  married. 2=single, 3= others)


In [3]:
# columns and rows
df.shape

(30000, 25)

In [4]:
# get information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [5]:
# get a list of the columns on the data
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [6]:
# unique IDs
df['ID'].nunique()

29687

In [7]:
id_counts = df['ID'].value_counts()
id_counts.head()

297edb0f-3bb1    2
f31d1048-5fa9    2
4cf8397f-6359    2
183f18d5-0b44    2
5deff6b6-62ff    2
Name: ID, dtype: int64

In [8]:
# show id counts categories, 1 count= unique, 2 or more= duplicate
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

#### Boolean Masks
- a way to filter an array or series bt some conditions also known as a logical mask.
- can use various conditional operators such as 'is equal to', '>', '<' etc.
- output is a series of True/False values i.e. boolean values.

In [9]:
# assign dupe_mask to id counts = to 2
dupe_mask = id_counts == 2
dupe_mask[0:10]

297edb0f-3bb1    True
f31d1048-5fa9    True
4cf8397f-6359    True
183f18d5-0b44    True
5deff6b6-62ff    True
733aa344-0dae    True
5718f600-12ac    True
a7559852-f405    True
a9870d3e-b4d1    True
ff8b7170-ce3a    True
Name: ID, dtype: bool

In [10]:
id_counts.index[0:10]

Index(['297edb0f-3bb1', 'f31d1048-5fa9', '4cf8397f-6359', '183f18d5-0b44',
       '5deff6b6-62ff', '733aa344-0dae', '5718f600-12ac', 'a7559852-f405',
       'a9870d3e-b4d1', 'ff8b7170-ce3a'],
      dtype='object')

In [11]:
# duplicate ids is equal to id counts with index= value fo dupe mask i.e. 2
dupe_ids = id_counts.index[dupe_mask]

In [12]:
# covert dupe ids to list and count ow many they are to determien how many values are duplicates
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [13]:
# first 10 duplicate rows
dupe_ids[0:10]

['297edb0f-3bb1',
 'f31d1048-5fa9',
 '4cf8397f-6359',
 '183f18d5-0b44',
 '5deff6b6-62ff',
 '733aa344-0dae',
 '5718f600-12ac',
 'a7559852-f405',
 'a9870d3e-b4d1',
 'ff8b7170-ce3a']

In [14]:
# selecting 3 duplicate IDs from the dataset
df.loc[df['ID'].isin(dupe_ids[0:3]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
17716,297edb0f-3bb1,120000,2,2,1,51,-1,-1,0,0,...,58677,51549,46552,62462,2299,1919,1838,2200,90000,0
17816,297edb0f-3bb1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25623,4cf8397f-6359,140000,2,3,1,48,0,0,-1,0,...,93325,94256,97009,0,139067,4263,3376,4274,4635,0
25723,4cf8397f-6359,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25930,f31d1048-5fa9,210000,2,2,2,30,0,0,0,0,...,215635,163741,162253,9500,9100,9900,6100,6100,6300,0
26030,f31d1048-5fa9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# boolen matrix same size as entire dataframe based on 'is equal to zero' condition
df_zero_mask = df == 0

In [16]:
# Boolean series feature_zero_mask, with all columns starting from index 1 having 0's
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [17]:
# sum of boolean series 'feature_zero_mask'
sum(feature_zero_mask)

315

In [18]:
# clean dataframe by eliminating the rows with all zeros except ids
df_clean_1 = df.loc[~feature_zero_mask,:].copy()

In [19]:
# verify the shape of new dataframe
df_clean_1.shape

(29685, 25)

In [20]:
# number of unique id, for verification
df_clean_1['ID'].nunique()

29685

In [21]:
# save the dataframe to CSV file
df_clean_1.to_csv('Data/df_clean_1.csv', index=False)